## Subword Tokenizer

기계에게 아무리 많은 단어를 학습시켜도, 세상의 모든 단어를 학습하는 것은 불가. <br>
만일 모르는 단어가 등장하면 그 단어를 단어 집합(corpus)에 없는 단어란 의미에서 OOV(Out-Of-Vocabulary) 또는 UNK(Unknown Token)라 표현 <br>

<br>

Subword segmenation은 하나의 단어는 더 작은 단위의 여러 단어들(toothpaste = tooth + paste)의 조합으로 구성된 경우가 많음. <br>
하나의 단어를 여러 서브워드로 분리해서 단어를 인코딩 및 임베딩하겠다는 의도를 가진 전처리 작업. <br>
이를 통해 OOV나 희귀 단어, 신조어와 같은 문제를 완화. <br>

### Byte Pair Encoding

reference: [Byte Pair Encoding](https://wikidocs.net/22592) <br>
paper: [Neural Machine Translation of Rare Words with Subword Units](https://arxiv.org/pdf/1508.07909.pdf)

![](https://wikidocs.net/images/page/22592/%EA%B7%B8%EB%A6%BC.png)


데이터 압축 알고리즘으로, 자연어에서 corpus를 구축할 때 사용하게 됨. <br>
글자 단위에서 점차 vocabulary를 만들어내는 bottom up 방식 <br>
데이터 내 모든 글자 단위로 vocabulary를 만들고, 가장 많이 등장하는 단어를 하나로 통합

<br>

<span style="font-size:16pt;"> Example </span>

최대 반복 횟수를 10회로 가정

> 단어 등장 횟수가 아래와 같음 <br>
> low : 5, lower : 2, newest : 6, widest : 3
> 
> 기존의 접근에서 vocabulary는 low, lower, newest, widest가 됨 <br>
> 만일 lowest라는 단어가 새로 등장하면 기존 vocabulary에 없는 OOV 발생
>
> BPE 사용 시 모든 단어를 글자 단위로 분리 <br>
> l o w: 5,  l o w e r: 2,  n e w e s t: 6,  w i d e s t: 3
> 1. e s는 newest에서 6번, widest에서 3번으로 총 9회 등장 <br>
> (l o, o w)는 5회 2회로 총 7회 등장
> 현 시점에서 vocabulary: l, o, w, e, r, n, s, t, i, d
> 2. e s를 es로 병합 <br>
> l o w: 5,  l o w e r: 2, n e w es t: 6, w i d es t: 3 <br>
> 현 시점에서 vocabulary: l, o, w, e, r, n, s, t, i, d, es <br>
> 3. es와 t를 결합 (9회 등장) <br>
> l o w: 5,  l o w e r: 2, n e w est: 6, w i d est: 3 <br>
> 현 시점에서 vocabulary: l, o, w, e, r, n, s, t, i, d, es, est <br>
> 4. l o를 lo로 병합 (7회 등장) <br>
> lo w: 5,  lo w e r: 2, n e w est: 6, w i d est: 3 <br>
> 현 시점에서 vocabulary: l, o, w, e, r, n, s, t, i, d, es, est, lo <br>
>
> 위의 작업을 10회 반복 시 아래와 같은 결론을 얻을 수 있음 <br>
> low: 5, low e r: 2, newest: 6, widest: 3 <br>
> l, o, w, e, r, n, s, t, i, d, es, est, lo, low, ne, new, newest, wi, wid, widest <br>

위의 경우 lowest라는 단어가 등장하면 low와 est를 결합하여 표현 가능 <br>
&nbsp;&nbsp;&nbsp;&nbsp;-> OOV가 아님

### SentencePiece

paper: [SentencePiece: A simple and language independent subword tokenizer
and detokenizer for Neural Text Processing](https://arxiv.org/pdf/1808.06226) <br>
github: [sentencepiece](https://github.com/google/sentencepiece)

<br>

내부 단어 분리를 위한 알고리즘. <br>

subword tokenizer를 사용하기 위해서 단어 토큰화를 먼저 진행한 상태여야 한다면, 이 subword 알고리즘을 모든 언어에 사용하는 것은 쉽지 않음. <br>
&nbsp;&nbsp;&nbsp;&nbsp;-> 한국어와 같은 언어는 단어 토큰화부터가 쉽지 않음. <br>
이런 사전 토큰화 작업없이 전처리를 하지 않은 raw data에 바로 단어 분리 토크나이저를 사용할 수 있다면, <br>
이 토크나이저는 그 어떤 언어에도 적용할 수 있는 토크나이저가 될 것. <br>
**sentencepiece는 사전 토큰화 작업없이 subword tokenization을 수행하므로 언어에 종속되지 않음** <br>

<br>


<span style="font-size:16pt;"> Example </span>

> 다음과 같은 문장 존재 <br>
> SentencePiece is simple.
> 
> 1. 모든 문장을 문자 단위로 쪼개고 공백도 하나의 문자로 취급 <br>
> S, e, n, t, e, n, c, e, _, P, i, e, c, e, _, i, s, _, s, i, m, p, l, e, . <br>
> 2. 각 문자의 빈도를 계산. 이때 빈도가 높은 문자 쌍을 병합. <br>
> ex) 'e'와 'n'의 빈도가 높다면 'en'으로 병합합니다. <br>
> S, en, t, en, c, e, _, P, i, e, c, e, _, i, s, _, s, i, m, p, l, e, .
> 3. 빈도가 높은 쌍을 계속 병합하여 원하는 vocab size가 될 때까지 반복 <br>
> ex) 'en' + 't' -> 'ent' <br>
> S, ent, ent, c, e, _, P, i, e, c, e, _, i, s, _, s, i, m, p, l, e, . <br>
> ex) 'i' + 's' -> 'is' <br> 
> S, ent, ent, c, e, _, P, i, e, c, e, _, is, _, s, i, m, p, l, e, . <br>